Берём натренированную VGG16, отрезаем голову и прогоняем её на большом кол-ве нагенеренных примеров (идея стырена отсюда: https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)

In [1]:
cd ..

/home/bfilippov/kaggle/whiteboard


In [2]:
import keras
import pandas as pd
import ast

train = pd.read_csv('source/train.csv')
test = pd.read_csv('source/test.csv')
train['labels'] = train['labels'].map(ast.literal_eval)
test['labels'] = test['labels'].map(ast.literal_eval)

valid = train.sample(frac=0.2, random_state=1313)
train = train.drop(valid.index)

Using TensorFlow backend.


In [4]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf

from keras.layers import Dense, Flatten, Input
from keras.models import Model
from keras import backend as K

from models.model_zoo import whiteboard_detector, whiteboard_loss
from keras.applications import VGG16
from models.image_generator import whiteboard_images

sess = tf.Session()
K.set_session(sess)

input_height = 224
input_width = 224

input_image = Input(shape=(input_height, input_width, 3))
resnet = VGG16(input_tensor=input_image, include_top=False)

for l in resnet.layers:
    l.trainable=False
    
resnet = Flatten()(resnet.output)

vgg16 = Model(input=input_image, output=resnet)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
img_dir = 'source/'
img_size = (input_height, input_width)

train_imgs = whiteboard_images(train, img_dir, img_size, batch_size=5, seed=1313)
valid_imgs = whiteboard_images(valid, img_dir, img_size, batch_size=5, seed=1313)
test_imgs = whiteboard_images(test, img_dir, img_size, batch_size=5, seed=1313)

In [8]:
import numpy as np
train_size = 10 * 1000
test_size = 250
valid_size = 250

def save_batch(fpath, generator, size, batch_size=50):
    f = open(fpath, 'w+b')
    for i in range(0, size, batch_size):
        batch = vgg16.predict_generator(generator, batch_size)
        np.save(f, batch)
        print("Processed", i, "images for", fpath)

save_batch('vgg16_train.npy', train_imgs, train_size)
print("Saved train data")

save_batch("vgg16_valid.npy", valid_imgs, valid_size)
print("Saved validation data")

save_batch("vgg16_test.npy", test_imgs, test_size)
print("Saved test data")
    

Processed 0 images for vgg16_train.npy
Processed 50 images for vgg16_train.npy
Processed 100 images for vgg16_train.npy
Processed 150 images for vgg16_train.npy
Processed 200 images for vgg16_train.npy
Processed 250 images for vgg16_train.npy
Processed 300 images for vgg16_train.npy
Processed 350 images for vgg16_train.npy
Processed 400 images for vgg16_train.npy
Processed 450 images for vgg16_train.npy
Processed 500 images for vgg16_train.npy
Processed 550 images for vgg16_train.npy
Processed 600 images for vgg16_train.npy
Processed 650 images for vgg16_train.npy
Processed 700 images for vgg16_train.npy
Processed 750 images for vgg16_train.npy
Processed 800 images for vgg16_train.npy
Processed 850 images for vgg16_train.npy
Processed 900 images for vgg16_train.npy
Processed 950 images for vgg16_train.npy
Processed 1000 images for vgg16_train.npy
Processed 1050 images for vgg16_train.npy
Processed 1100 images for vgg16_train.npy
Processed 1150 images for vgg16_train.npy
Processed 1200 